In [1]:
import h5py
import numpy as np
import glob
import json
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
iogd={'1':[0,0],'2':[0,1],'3':[0,2],'4':[0,3],
      '5':[1,0],'6':[1,1],'7':[1,2],'8':[1,3]}

In [ ]:
filedir='/global/cfs/cdirs/dune/www/data/2x2/nearline/packet/beam/july10_2024/nominal_hv/'
map_q_vs_beam_dt(filedir, ['0050018'], False, False)
filedir='/global/cfs/cdirs/dune/www/data/2x2/nearline/packet/beam/july8_2024/nominal_hv/'
map_q_vs_beam_dt(filedir, ['0050017'], False, False)
filedir='/global/cfs/cdirs/dune/www/data/2x2/nearline/packet/beam/july7_2024/half_nominal_hv/'
map_q_vs_beam_dt(filedir, ['0050011'], False, False)

In [3]:
def map_q_vs_beam_dt(filedir, run_no, is_mc, plot_file):
    
    
    
    for filename in tqdm(glob.glob(filedir+'*.h5')):
        if is_mc==False:
            file_run_no=filename.split("/")[-1].split("-")[1]
            if file_run_no not in run_no: continue
            dt_string=filename.split("/")[-1].split("-")[2].split("_CDT")[0]    

        f = h5py.File(filename,'r')
        packets=f['packets']
        ext_mask=((packets['packet_type']==7)&(packets['io_group']==5))
        exts=packets[ext_mask]
        ext_idcs=np.argwhere(ext_mask).flatten()
        message_groups = np.split(packets, ext_idcs)
        
        out={}
        for i in range(1,9,1):
            dumbx=[]; dumby=[]
            n, xedge, yedge = np.histogram2d(dumbx, dumby, bins=[201,256], range=[[0,200],[0,256]] )
            out[str(i)]=n
        
        for mg in message_groups:
            trig_mask=((mg['packet_type']==7)&(mg['io_group']==5))
            trig_timestamps=mg[trig_mask]['timestamp'] # LArPix MCLK
            if len(trig_timestamps)==0: continue

            for i in range(1,9,1):
                data_mask=((mg['packet_type']==0)&(mg['valid_parity']==1)&(mg['io_group']==i))
                timestamp_mask=((mg['timestamp']>=trig_timestamps[0])&(mg['timestamp']<=trig_timestamps[0]+2e3))
                mask=np.logical_and(data_mask, timestamp_mask)
                tpc_packets=mg[mask]
                n, xedge, yedge = np.histogram2d([i-trig_timestamps[0] for i in tpc_packets['timestamp']], tpc_packets['dataword'], bins=[201,256], range=[[0,2000],[0,256]])
                out[str(i)]+=n
        f.close()
        
        if plot_file==True:
            fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(24,12))
            for iog in out.keys():
                x=[0]*300; y=[0]*300
                for j in range(len(out[iog])):
                    for k in range(len(out[iog][j])):
                        if out[iog][j][k]==0: continue
                        for l in range(int(out[iog][j][k])): x.append(j); y.append(k)
                h = ax[iogd[iog][0]][iogd[iog][1]].hist2d(x,y, bins=[np.linspace(0,200,201), np.linspace(0,255,256)], norm=LogNorm())
            fig.colorbar(h[3], ax=ax[iogd[iog][0]][iogd[iog][1]])
            plt.show()
        else:
            for key in out.keys(): out[key]=out[key].tolist()
            if is_mc==True:
                with open('q_vs_dt_mc_'+file_run_no+'-'+dt_string+'.json','w') as outfile: json.dump(out, outfile, indent=4)
            if is_mc==False:
                with open('q_vs_dt_'+file_run_no+'-'+dt_string+'.json','w') as outfile: json.dump(out, outfile, indent=4)      